In [20]:
import json
import csv
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from datetime import datetime
import re
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import tensorflow_text as text

In [6]:
pd.options.display.max_colwidth =170

In [2]:
train = pd.read_json(r"C:\Users\risha\Desktop\convolve-epoch1\train.json",typ = 'dataframe').to_frame('count')

In [3]:
train.reset_index(inplace = True)
train.rename(columns = {'index':'log'}, inplace = True)

In [4]:
train['log'] = train['log'].map(lambda x: x.rstrip('\n'))

In [9]:
train

,log,count
0,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected,normal
1,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.527847 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected,normal
2,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.675872 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected,normal
3,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.823719 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected,normal
4,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.982731 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected,normal
...,...,...
4152654,KERNDTLB 1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-20.05.30.386651 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt,anomaly
4152655,KERNDTLB 1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-20.05.30.558220 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt,anomaly
4152656,KERNDTLB 1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-20.05.30.675784 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt,anomaly
4152657,KERNDTLB 1118545530 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-20.05.30.777307 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt,anomaly


In [61]:
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
for i in range(4152659):
    l1.append(df.log[i].split(' ')[1])
    l2.append(df.log[i].split(' ')[2])
    l3.append(df.log[i].split(' ')[3])
    l4.append(df.log[i].split(' ')[4])
    l5.append(df.log[i].split(' ')[5])
    l6.append(' '.join(df.log[i].split(' ')[6:]))
    l7.append(df['count'][i])

In [62]:
data = pd.DataFrame(list(zip(l1,l2,l3,l4,l5,l6,l7)))

In [64]:
data

,0,1,2,3,4,5,6
0,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.363779,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
1,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.527847,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
2,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.675872,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
3,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.823719,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
4,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.982731,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
...,...,...,...,...,...,...,...
4152654,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.386651,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly
4152655,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.558220,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly
4152656,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.675784,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly
4152657,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.777307,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly


In [21]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [23]:
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [24]:
def get_sentence_embedding(sentences):
    preprocessed_text = preprocessor(sentences)
    return encoder(preprocessed_text)['pooled_output']

In [26]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'text')
preprocessed_text = preprocessor(text_input)
outputs = encoder(preprocessed_text)

# nn layers

l = tf.keras.layers.Dropout(0.1, name = 'dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output')(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [28]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
    tf.keras.metrics.Precision(name = 'precision'),
    tf.keras.metrics.Recall(name = 'recall')
    
]

In [29]:
model.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics = METRICS)

In [31]:
data

,0,1,2,3,4,5,6
0,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.363779,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
1,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.527847,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
2,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.675872,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
3,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.823719,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
4,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.982731,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error corrected,normal
...,...,...,...,...,...,...,...
4152654,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.386651,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly
4152655,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.558220,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly
4152656,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.675784,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly
4152657,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.777307,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly


In [32]:
l=[]
for i in data[6]:
    if(i=='normal'):
        l.append(1)
    else:
        l.append(0)

In [33]:
data['target'] = l

In [41]:
df1 = data[data['target']==1].sample(n = 1000000)

In [42]:
df2 = data[data['target']==0]

In [43]:
lmao = pd.concat([df1,df2])

In [77]:
lmao

,0,1,2,3,4,5,6,target
1813632,1121252825,2005.07.13,R34-M0-N2-C:J17-U01,2005-07-13-04.07.05.268548,R34-M0-N2-C:J17-U01,RAS KERNEL INFO 114684368 double-hummer alignment exceptions,normal,1
417693,1118771268,2005.06.14,R20-M1-N8-C:J03-U01,2005-06-14-10.47.48.856501,R20-M1-N8-C:J03-U01,RAS KERNEL FATAL program interrupt: unimplemented operation..0,normal,1
3493767,1131060202,2005.11.03,R67-M1-NF-C:J13-U11,2005-11-03-15.23.22.459222,R67-M1-NF-C:J13-U11,RAS KERNEL INFO iar 00106568 dear 02494bbc,normal,1
620374,1119151542,2005.06.18,R13-M1-NF-C:J03-U11,2005-06-18-20.25.42.550852,R13-M1-NF-C:J03-U11,RAS KERNEL INFO generating core.11693,normal,1
495782,1118772603,2005.06.14,R27-M1-ND-C:J16-U01,2005-06-14-11.10.03.301532,R27-M1-ND-C:J16-U01,RAS KERNEL FATAL 20:00000000 21:1feaa7b0 22:00000000 23:00000000,normal,1
...,...,...,...,...,...,...,...,...
4152654,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.386651,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly,0
4152655,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.558220,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly,0
4152656,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.675784,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly,0
4152657,1118545530,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-20.05.30.777307,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt,anomaly,0


In [47]:
X_train = lmao[5]
y_train = lmao['target']

In [49]:
test = pd.read_csv(r"C:\Users\risha\Desktop\convolve-epoch1\test.csv")

In [53]:
test[' Log'] = test[' Log'].map(lambda x: x.rstrip('\n'))

In [57]:
test.drop([' Log'], axis = 1, inplace = True)

In [66]:
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
for i in range(len(test)):
    l1.append(test.Log[i].split(' ')[1])
    l2.append(test.Log[i].split(' ')[2])
    l3.append(test.Log[i].split(' ')[3])
    l4.append(test.Log[i].split(' ')[4])
    l5.append(test.Log[i].split(' ')[5])
    l6.append(' '.join(test.Log[i].split(' ')[6:]))
    

In [67]:
data_test = pd.DataFrame(list(zip(l1,l2,l3,l4,l5,l6)))

In [68]:
data_test

,0,1,2,3,4,5
0,1124336301,2005.08.17,R13-M1-N8-C:J12-U01,2005-08-17-20.38.21.466368,R13-M1-N8-C:J12-U01,RAS KERNEL FATAL rts: kernel terminated for reason 1001
1,1118553175,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-22.12.55.707149,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt
2,1118536033,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-17.27.13.042387,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt
3,1117992566,2005.06.05,R30-M1-N6-C:J03-U11,2005-06-05-10.29.26.943462,R30-M1-N6-C:J03-U11,RAS KERNEL INFO generating core.6463
4,1118538965,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-18.16.05.049256,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt
...,...,...,...,...,...,...
595295,1118768234,2005.06.14,R20-M0-NA-C:J17-U11,2005-06-14-09.57.14.709136,R20-M0-NA-C:J17-U11,RAS KERNEL FATAL exception syndrome register: 0x00800000
595296,1118370947,2005.06.09,R15-M0-NA-C:J02-U01,2005-06-09-19.35.47.353971,R15-M0-NA-C:J02-U01,RAS KERNEL INFO generating core.10895
595297,1118540242,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-18.37.22.743936,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt
595298,1118538947,2005.06.11,R30-M0-N9-C:J16-U01,2005-06-11-18.15.47.247097,R30-M0-N9-C:J16-U01,RAS KERNEL FATAL data TLB error interrupt


In [69]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk

In [70]:
vocab_size = 10000
embedding_dim = 16
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [71]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token = oov_tok)
tokenizer.fit_on_texts(lmao[5])
word_index = tokenizer.word_index

In [72]:
word_index

{'<OOV>': 1,
 'ras': 2,
 'kernel': 3,
 'info': 4,
 'core': 5,
 'generating': 6,
 '0': 7,
 'fatal': 8,
 'error': 9,
 'alignment': 10,
 'exceptions': 11,
 'iar': 12,
 'interrupt': 13,
 'dear': 14,
 'data': 15,
 'floating': 16,
 'double': 17,
 'hummer': 18,
 'point': 19,
 'tlb': 20,
 'ciod': 21,
 'program': 22,
 'is': 23,
 'mask': 24,
 'at': 25,
 'ce': 26,
 'sym': 27,
 'file': 28,
 'or': 29,
 'microseconds': 30,
 'source': 31,
 'assert': 32,
 'idotransportmgr': 33,
 'input': 34,
 'instruction': 35,
 'corrected': 36,
 'app': 37,
 'critical': 38,
 '1': 39,
 'interrupts': 40,
 'a': 41,
 'the': 42,
 'for': 43,
 'in': 44,
 'no': 45,
 'total': 46,
 'address': 47,
 'node': 48,
 'ddr': 49,
 'not': 50,
 'detected': 51,
 '5': 52,
 'invalid': 53,
 'card': 54,
 'of': 55,
 'null': 56,
 'missing': 57,
 'on': 58,
 'image': 59,
 'loading': 60,
 'bgl': 61,
 'home': 62,
 'discovery': 63,
 'cache': 64,
 's': 65,
 'line': 66,
 '15': 67,
 'mmcs': 68,
 'time': 69,
 'spent': 70,
 'parity': 71,
 'directory': 72,

In [73]:
sequences = tokenizer.texts_to_sequences(lmao[5])
X_train_final = pad_sequences(sequences,maxlen = max_length,truncating = trunc_type)
testing_sentences = tokenizer.texts_to_sequences(data_test[5])
X_test_final = pad_sequences(testing_sentences,maxlen = max_length,truncating = trunc_type)

In [74]:
# ayushi nn
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [75]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

In [76]:
y_train = lmao['target']

1813632    1
417693     1
3493767    1
620374     1
495782     1
          ..
4152654    0
4152655    0
4152656    0
4152657    0
4152658    0
Name: target, Length: 1069692, dtype: int64

In [78]:
model.fit(X_train_final,y_train,epochs = 2)

Epoch 1/2
33428/33428 [==============================] - 111s 3ms/step - loss: 0.0036 - accuracy: 0.9992 - precision: 0.9993 - recall: 0.9999
Epoch 2/2
33428/33428 [==============================] - 109s 3ms/step - loss: 1.0785e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000


In [79]:
y_pred = model.predict(X_test_final)

18604/18604 [==============================] - 21s 1ms/step


In [80]:
y_pred = y_pred.flatten()

In [81]:
y_pred_final = []
for i in range(len(y_pred)):
    if(y_pred[i]>0.5):
        y_pred_final.append('normal')
    else:
        y_pred_final.append('anomaly')

In [82]:
y_pred_final = np.array(y_pred_final)

In [83]:
final = pd.DataFrame({'ID':test['ID'],'Label':y_pred_final})

In [88]:
final

,ID,Label
0,0,normal
1,1,anomaly
2,2,anomaly
3,3,normal
4,4,anomaly
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,anomaly
595298,595298,anomaly


In [86]:
test.iloc[45000]

ID                                                                                                                              45000
Log     1117989459 2005.06.05 R37-M1-N6-C:J07-U01 2005-06-05-09.37.39.233896 R37-M1-N6-C:J07-U01 RAS KERNEL INFO generating core.4534
Name: 45000, dtype: object

In [89]:
final.to_csv('lets.csv')